## Парсер и предобработка

In [1]:
! pip install fake_useragent -q

In [2]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import time
from datetime import datetime
import random
import re

session = requests.session()

# используем фейк юзерагент, чтобы нас не поймали за руку
from fake_useragent import UserAgent

ua = UserAgent()

url = 'https://scarykids.ru/?dir=/pioneer'
r = session.get(url, headers={'User-Agent': ua.random})
soup = BeautifulSoup(r.content)
text_li = soup.find_all('li') # находим строчки с тегом li
links = []
for link in text_li:
  links.append(link.find('a').attrs['href']) # собираем список ссылок

In [4]:
story_dict = {} # формат название : текст

# проходимся по списку ссылок
for link in links:
  url = f'https://scarykids.ru{link}'
  r = session.get(url, headers={'User-Agent': ua.random})
  soup = BeautifulSoup(r.content)

  hr_tag = soup.find('hr')

  # достаём заголовок
  title = soup.find('h1').text

  story_text = ""
  for elem in hr_tag.next_siblings:
      if elem.name in ['table', 'script', 'div']:
          break
      if isinstance(elem, str):
          story_text += elem
      else:
          story_text += elem.get_text(separator="\n").strip()

  # достаём текст истории
  story_text = story_text.replace('\r', ' ').strip()

  # записываем в словарь
  story_dict[title] = story_text

In [7]:
import pandas as pd

# создаём пандас из словаря
stories = pd.DataFrame(list(story_dict.items()), columns=['название', 'текст'])

In [8]:
stories

,название,текст
0,Красное печенье,"У одних девочек была мама, а папы не было. И к..."
1,Красное печенье - 2,Один мужчина женился на женщине. У них родился...
2,Красное пятно,Одна семья получила новую квартиру. А там было...
3,Зеленые глаза,В одном городе жила девочка. У нее была бабушк...
4,Зеленые глаза - 2,Жили-были мать и дочка. Пошли они в магазин. Д...
...,...,...
287,Красивая цепочка,"Однажды мама с девочкой пошли в магазин, долго..."
288,Огненный приют,Однажды мужчина ехал ночью в Лунинец на своей ...
289,Вызов гнома,В одном лагере девочки в отряде умели плохо ри...
290,Сердце мамы,"Жила-была мать, у нее рано умер муж, и остался..."


In [9]:
import re

# функция очистки от лишних пробелов и табов
def clean_text(text):
    # удаляем двойные пробелы и заменяем табуляции на одиночные пробелы
    text = re.sub(r'\s+', ' ', text) # заменяет все виды пробельных символов (включая табуляции) одним пробелом
    return text.strip()

In [10]:
stories['текст'] = stories['текст'].apply(clean_text)

In [11]:
# сохраняем получившийся датасет в csv-файл
stories.to_csv('scary_stories.csv', index=False)

## Перевод .csv в conllu-датасет

In [14]:
! python -m spacy download ru_core_news_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 59.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [15]:
import spacy

# загружаем русскую модель
nlp = spacy.load('ru_core_news_sm')

In [18]:
import pandas as pd

# читаем csv-файл
stories = pd.read_csv("scary_stories.csv")

In [19]:
# функция для перевода в формат conllu
def to_conllu_format(doc, text_title):
    conllu_output = []
    sent_id = 0

    # проходимся по предложениям
    for sent in doc.sents:
        sent_id += 1
        # добавляем метаданные с названием, номером предложения и текстом предложения
        conllu_output.append(f"# text_title = {text_title}")
        conllu_output.append(f"# sent_id = {sent_id}")
        conllu_output.append(f"# text = {sent}")

        # проходимся по токенам в предложении
        for token in sent:
            # собираем данные для формата CoNLL-U
            token_id = token.i - sent.start + 1
            form = token.text
            lemma = token.lemma_
            upos = token.pos_
            xpos = "_"  # нет данных для XPOS, оставляем "_"
            feats = "|".join([f"{k}={v}" for k, v in token.morph.to_dict().items()]) if token.morph else "_"
            head = token.head.i - sent.start + 1 if token.head != token else 0
            deprel = token.dep_
            deps = "_"  # дополнительные зависимости отсутствуют
            misc = "_"  # можно использовать для дополнительных данных, оставляем пустым

            # форматируем строку для каждого токена
            conllu_line = f"{token_id}\t{form}\t{lemma}\t{upos}\t{xpos}\t{feats}\t{head}\t{deprel}\t{deps}\t{misc}"
            conllu_output.append(conllu_line)

        conllu_output.append("")  # пустая строка между предложениями

    return "\n".join(conllu_output)

In [20]:
# записываем в файл
with open ('conllu_stories.conllu', 'w') as my_file:
  # проходимся циклом по датасету
  for index, row in stories.iterrows():
    text = row['текст']
    text_title = row['название']
    doc = nlp(text)
    # и инициализируем функцию на каждую пару текст + название
    my_file.write(to_conllu_format(doc, text_title))